In [271]:
from collections import Counter
import operator
import re
import psycopg2 as pg2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, probplot
import pylab
import statsmodels.stats.api as sms

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from sklearn.metrics.pairwise import cosine_similarity
#import nlp

In [7]:
conn = pg2.connect(dbname='dsjob', host='localhost',
                    password='docker', user='postgres')
conn.autocommit = True
cur = conn.cursor()

cur.execute("SELECT * FROM indeed;")
l=[]
for item in cur.fetchall():
    l.append(item)

df=pd.DataFrame(l, columns=["job_jk", "job_title",  "job_location",  "company_name", "salary", "description"])

In [8]:
df.head()

,job_jk,job_title,job_location,company_name,salary,description
0,1bce25f0b7e3723d,"Senior Data Scientist – Image Analytics, Novar...","Cambridge, MA",Novartis,No salary given,\n\nWe believe the answers are found when curi...
1,1486880d275ce7c1,Senior Scientist Deep Data Analytics,"Cambridge, MA 02142",Sanofi,No salary given,Sanofi is a global life sciences company commi...
2,7d6ac302ac846400,Senior Manager of Data Science – Cyber Security,"Boston, MA 02298",No company info,No salary given,\nPrimary Responsibilities:\n Product ownershi...
3,3100f3d85650c469,Enterprise Architect Data,"Boston, MA",MassMutual,No salary given,\nA career with us means you will work alongsi...
4,3de16480e89bcaa2,Post Doctoral Fellow,"Cambridge, MA",Eisai,No salary given,"\n\nAt G2D2, we focus not only on how such dat..."


In [9]:
vectorizer = TfidfVectorizer(stop_words = "english", max_features = None)
X = vectorizer.fit_transform(df['description'])
nmf = NMF(n_components=10, random_state =0)
W= nmf.fit_transform(X)
H = nmf.components_

In [10]:
W.shape, H.shape

((14473, 10), (10, 48220))

In [21]:
vocab = vectorizer.get_feature_names()
words_per_topic = []
for i in range(H.shape[0]):
    l=[]
    for e in H[i].argsort()[::-1][0:10]:
        l.append(vocab[e])
    print(l)
    words_per_topic.append(l)
#words_per_topic    

['learning', 'machine', 'data', 'models', 'algorithms', 'ai', 'ml', 'experience', 'science', 'deep']
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'digital', 'teams', 'experience']
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
['software', 'experience', 'cloud', 'systems', 'aws', 'development', 'design', 'technical', 'data', 'technologies']
['accenture', 'cloud', 'data', 'client', 'azure', 'visa', 'clients', 'delivery', 'gcp', 'employment']
['deloitte', 'clients', 'people', 'cognitive', 'business', 'client', 'tips', 'learn', 'help', 'offer']
['iqvia', 'healthcare', 'takes', 'clinical', 'reasonable', 'accommodation', 'patients', 'clients', 'health', 'client']
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']
['status', 'employment', 'microsoft', 'protected', 'qualified', 'applicants', 'disability', 'gender', 'information', 'national']
[

In [25]:
def max_feature_topic_search(m_feature_n, topic_n):
    for m in m_feature_n:
        for n in topic_n:
            vectorizer = TfidfVectorizer(stop_words = "english", max_features = m)
            X = vectorizer.fit_transform(df['description'])
            nmf = NMF(n_components=n, random_state =0)
            W= nmf.fit_transform(X)
            H = nmf.components_
            vocab = vectorizer.get_feature_names()
            words_per_topic = []
            print ("max features = {}, n of topics ={}".format(m, n))
            for i in range(H.shape[0]):
                l=[]
                for e in H[i].argsort()[::-1][0:10]:
                    l.append(vocab[e])
                print("topic {}".format(i))
                print(l)
                words_per_topic.append(l)
    

In [26]:
m_feature_n =[20000, 10000, 5000, 2000, 1000]
topic_n = [40, 20, 10, 8, 6, 4]

max_feature_topic_search(m_feature_n, topic_n)

max features = 20000, n of topics =40
topic 0
['reporting', 'bi', 'business', 'data', 'reports', 'analysis', 'ability', 'dashboards', 'tableau', 'sql']
topic 1
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'application', 'workday_recruiting', 'brave', 'arrange', 'ultimately']
topic 2
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'school', 'date']
topic 3
['software', 'systems', 'development', 'engineering', 'engineers', 'test', 'code', 'engineer', 'technical', 'infrastructure']
topic 4
['accenture', 'visa', 'client', 'clients', 'employment', 'authorization', 'status', 'delivery', 'technology', 'business']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'tips', 'learn', 'help', 'confident', 'offer', 'client']
topic 6
['learning', 'machine', 'ai', 'ml', 'algorithms', 'deep', 'models', 'science', 'nlp', 'tensorflow']
topic 7
['intelligence', 'security', 'll', 'clearance', 'warfighters', 'abroad', 'information',

max features = 20000, n of topics =8
topic 0
['data', 'business', 'analytics', 'experience', 'analysis', 'models', 'statistical', 'modeling', 'science', 'tools']
topic 1
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'strategy', 'teams', 'management']
topic 2
['research', 'university', 'teaching', 'faculty', 'health', 'position', 'department', 'students', 'information', 'program']
topic 3
['experience', 'software', 'cloud', 'learning', 'aws', 'amazon', 'systems', 'machine', 'engineering', 'development']
topic 4
['accenture', 'cloud', 'data', 'azure', 'client', 'visa', 'employment', 'delivery', 'clients', 'status']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'client', 'learn', 'business', 'tips', 'help', 'offer']
topic 6
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'clients', 'health', 'client']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']


max features = 10000, n of topics =10
topic 0
['business', 'management', 'ability', 'project', 'analytics', 'skills', 'experience', 'financial', 'analysis', 'work']
topic 1
['product', 'marketing', 'team', 'customer', 'sales', 'products', 'customers', 'business', 'work', 'teams']
topic 2
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
topic 3
['software', 'experience', 'cloud', 'systems', 'development', 'aws', 'design', 'technical', 'services', 'security']
topic 4
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'sales', 'health', 'clients']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'learn', 'tips', 'client', 'help', 'offer', 'confident']
topic 6
['learning', 'machine', 'models', 'algorithms', 'experience', 'ml', 'ai', 'techniques', 'statistical', 'data']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'information', 'dat

max features = 5000, n of topics =20
topic 0
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'reporting', 'risk', 'amp']
topic 1
['team', 'work', 'll', 'company', 'people', 'help', 'world', 'best', 'make', 'insurance']
topic 2
['university', 'research', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'position']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'technical', 'computer', 'web']
topic 4
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'clients', 'application', 'workday_recruiting', 'brave', 'client']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'tips', 'client', 'learn', 'help', 'confident', 'offer']
topic 6
['learning', 'machine', 'models', 'algorithms', 'techniques', 'statistical', 'experience', 'data', 'ml', 'ai']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'change', 'sci']
to

max features = 2000, n of topics =20
topic 0
['data', 'experience', 'analytics', 'sql', 'science', 'sets', 'tools', 'big', 'sources', 'analysis']
topic 1
['university', 'teaching', 'faculty', 'students', 'research', 'department', 'edu', 'undergraduate', 'graduate', 'college']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'best', 'world', 'make', 'build']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'computer', 'web', 'technical']
topic 4
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'risk', 'reporting', 'amp']
topic 5
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'patients', 'application', 'clients', 'workday_recruiting', 'brave']
topic 6
['status', 'employment', 'protected', 'gender', 'qualified', 'microsoft', 'disability', 'applicants', 'origin', 'race']
topic 7
['deloitte', 'clients', 'people', 'cognitive', 'client', 'tips', 'learn', 'help', 'confident', 'offer']

max features = 1000, n of topics =20
topic 0
['data', 'big', 'science', 'pipelines', 'sql', 'analytics', 'etl', 'tools', 'sets', 'sources']
topic 1
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'graduate', 'undergraduate', 'college', 'position']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'world', 'best', 'make', 'love']
topic 3
['software', 'experience', 'development', 'design', 'systems', 'engineering', 'code', 'test', 'web', 'testing']
topic 4
['learning', 'machine', 'ai', 'models', 'ml', 'algorithms', 'deep', 'science', 'data', 'techniques']
topic 5
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'application', 'patients', 'clients', 'impact', 'disabled']
topic 6
['status', 'employment', 'gender', 'protected', 'disability', 'qualified', 'applicants', 'origin', 'race', 'religion']
topic 7
['deloitte', 'clients', 'people', 'client', 'learn', 'help', 'offer', 'professionals', 'culture', 'communities']
topic 8
['statistical', 'an

In [107]:
token = nltk.word_tokenize(" ".join(df['description'].map(str)).lower())



In [109]:
len(token)

9170386

In [108]:
lemmatizer = WordNetLemmatizer()

description_lemma = [lemmatizer.lemmatize(e) for e in token]

In [110]:
len(description_lemma)
description_lemma[0:100]

['we',
 'believe',
 'the',
 'answer',
 'are',
 'found',
 'when',
 'curious',
 ',',
 'courageous',
 'and',
 'collaborative',
 'people',
 'like',
 'you',
 'are',
 'brought',
 'together',
 'in',
 'an',
 'inspiring',
 'environment',
 '.',
 'where',
 'you',
 '’',
 're',
 'given',
 'opportunity',
 'to',
 'explore',
 'the',
 'power',
 'of',
 'digital',
 'and',
 'data',
 '.',
 'where',
 'you',
 '’',
 're',
 'empowered',
 'to',
 'risk',
 'failure',
 'by',
 'taking',
 'smart',
 'risk',
 ',',
 'and',
 'where',
 'you',
 '’',
 're',
 'surrounded',
 'by',
 'people',
 'who',
 'share',
 'your',
 'determination',
 'to',
 'tackle',
 'the',
 'world',
 '’',
 's',
 'toughest',
 'medical',
 'challenge',
 '.',
 'we',
 'are',
 'novartis',
 '.',
 'join',
 'u',
 'and',
 'help',
 'u',
 're-imagine',
 'medicine',
 '.',
 'two',
 'company',
 'and',
 'one',
 'incredible',
 'alliance',
 '.',
 'novartis',
 'and',
 'microsoft',
 'have',
 'formed',
 'alliance',
 'to',
 'leverage']

In [165]:
def tokenize_lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    token = nltk.word_tokenize(" ".join(df['description'].map(str)).lower())
    return [lemmatizer.lemmatize(e) for e in token]
    
    
tokenize_lemmatize(" ".join(df['description'].map(str)).lower())

['we',
 'believe',
 'the',
 'answer',
 'are',
 'found',
 'when',
 'curious',
 ',',
 'courageous',
 'and',
 'collaborative',
 'people',
 'like',
 'you',
 'are',
 'brought',
 'together',
 'in',
 'an',
 'inspiring',
 'environment',
 '.',
 'where',
 'you',
 '’',
 're',
 'given',
 'opportunity',
 'to',
 'explore',
 'the',
 'power',
 'of',
 'digital',
 'and',
 'data',
 '.',
 'where',
 'you',
 '’',
 're',
 'empowered',
 'to',
 'risk',
 'failure',
 'by',
 'taking',
 'smart',
 'risk',
 ',',
 'and',
 'where',
 'you',
 '’',
 're',
 'surrounded',
 'by',
 'people',
 'who',
 'share',
 'your',
 'determination',
 'to',
 'tackle',
 'the',
 'world',
 '’',
 's',
 'toughest',
 'medical',
 'challenge',
 '.',
 'we',
 'are',
 'novartis',
 '.',
 'join',
 'u',
 'and',
 'help',
 'u',
 're-imagine',
 'medicine',
 '.',
 'two',
 'company',
 'and',
 'one',
 'incredible',
 'alliance',
 '.',
 'novartis',
 'and',
 'microsoft',
 'have',
 'formed',
 'alliance',
 'to',
 'leverage',
 'data',
 '&',
 'amp',
 ';',
 'artifici

In [111]:
len(set(description_lemma))

74435

In [112]:
vocab_lem = set(description_lemma)

In [151]:
my_stop_words = stopwords.words('english')
len(my_stop_words)


179

In [152]:
my_stop_words.extend(list(string.punctuation))
my_stop_words.extend(['--'])
len(my_stop_words)

212

In [163]:
def n_grams_freq(tokens, n, stopwords):
    n_grams = ngrams(tokens,n)
    n_grams_sort = sorted(Counter(n_grams).items(), key = operator.itemgetter(1), reverse = True)
    
    return ( [(k,v) for k, v in n_grams_sort if all([e not in stopwords for e in k])] )

In [167]:
ngrams_2 = n_grams_freq(description_lemma,2, my_stop_words)
ngrams_3 = n_grams_freq(description_lemma,3, my_stop_words)
ngrams_4 = n_grams_freq(description_lemma,4, my_stop_words)

In [168]:
len(ngrams_2)

453439

In [169]:
len(ngrams_3)

469593

In [170]:
len(ngrams_4)

267015

In [176]:
ngrams_4[0:4000]

[(('equal', 'opportunity/affirmative', 'action', 'employer'), 525),
 (('client', 'drive', 'healthcare', 'forward'), 416),
 (('inc.', 'provides', 'reasonable', 'accommodation'), 379),
 (('human', 'health', 'take', 'insight'), 373),
 (('best', 'work', 'every', 'day'), 319),
 (('san', 'francisco', 'fair', 'chance'), 314),
 (('francisco', 'fair', 'chance', 'ordinance'), 314),
 (('equal', 'opportunity', 'employer', '–'), 298),
 (('equal', 'opportunity-affirmative', 'action', 'employer'), 295),
 (('opportunity', 'employer', '–', 'minority'), 283),
 (('opportunity-affirmative', 'action', 'employer', '–'), 277),
 (('action', 'employer', '–', 'minority'), 271),
 (('protected', 'characteristic—to', 'fearlessly', 'drive'), 252),
 (('characteristic—to', 'fearlessly', 'drive', 'change'), 252),
 (('also', 'consider', 'qualified', 'applicant'), 248),
 (('provides', 'equal', 'employment', 'opportunity'), 244),
 (('consider', 'qualified', 'applicant', 'regardless'), 239),
 (('equal', 'employment', 'opp

In [261]:
ngrams_3[0:150]

[(('equal', 'opportunity', 'employer'), 4046),
 (('employment', 'without', 'regard'), 2319),
 (('bachelor', "'s", 'degree'), 1788),
 (('protected', 'veteran', 'status'), 1646),
 (('equal', 'employment', 'opportunity'), 1573),
 (('data', 'science', 'team'), 1550),
 (('written', 'communication', 'skill'), 1400),
 (('large', 'data', 'set'), 1014),
 (('natural', 'language', 'processing'), 935),
 (('year', '’', 'experience'), 921),
 (('machine', 'learning', 'model'), 886),
 (('verbal', 'communication', 'skill'), 879),
 (('subject', 'matter', 'expert'), 861),
 (('master', "'s", 'degree'), 834),
 (('machine', 'learning', 'algorithm'), 753),
 (('machine', 'learning', 'technique'), 743),
 (('proven', 'track', 'record'), 677),
 (('opportunity/affirmative', 'action', 'employer'), 594),
 (('affirmative', 'action', 'employer'), 590),
 (('equal', 'opportunity/affirmative', 'action'), 571),
 (('excellent', 'communication', 'skill'), 558),
 (('employer', '–', 'minority'), 556),
 (('big', 'data', 'tech

In [263]:
ngrams_2[0:1000]

[(('data', 'science'), 19085),
 (('machine', 'learning'), 12113),
 (('computer', 'science'), 7092),
 (('data', 'scientist'), 6628),
 (('sexual', 'orientation'), 6062),
 (('national', 'origin'), 5497),
 (('communication', 'skill'), 5262),
 (('gender', 'identity'), 5166),
 (('equal', 'opportunity'), 5126),
 (('big', 'data'), 4895),
 (('opportunity', 'employer'), 4554),
 (('veteran', 'status'), 4527),
 (('experience', 'working'), 4112),
 (('without', 'regard'), 3538),
 (('data', 'analysis'), 3459),
 (('related', 'field'), 3251),
 (('qualified', 'applicant'), 3197),
 (('data', 'set'), 3191),
 (('team', 'member'), 3125),
 (('best', 'practice'), 3093),
 (('bachelor', '’'), 2921),
 (('data', 'analytics'), 2693),
 (("'s", 'degree'), 2677),
 (('software', 'development'), 2602),
 (('data', 'management'), 2453),
 (('work', 'experience'), 2389),
 (('marital', 'status'), 2384),
 (('employment', 'without'), 2351),
 (('reasonable', 'accommodation'), 2323),
 (('data', 'visualization'), 2272),
 (('busi

In [259]:
ngrams_3[:100]

[(('equal', 'opportunity', 'employer'), 4046),
 (('employment', 'without', 'regard'), 2319),
 (('bachelor', "'s", 'degree'), 1788),
 (('protected', 'veteran', 'status'), 1646),
 (('equal', 'employment', 'opportunity'), 1573),
 (('data', 'science', 'team'), 1550),
 (('written', 'communication', 'skill'), 1400),
 (('large', 'data', 'set'), 1014),
 (('natural', 'language', 'processing'), 935),
 (('year', '’', 'experience'), 921),
 (('machine', 'learning', 'model'), 886),
 (('verbal', 'communication', 'skill'), 879),
 (('subject', 'matter', 'expert'), 861),
 (('master', "'s", 'degree'), 834),
 (('machine', 'learning', 'algorithm'), 753),
 (('machine', 'learning', 'technique'), 743),
 (('proven', 'track', 'record'), 677),
 (('opportunity/affirmative', 'action', 'employer'), 594),
 (('affirmative', 'action', 'employer'), 590),
 (('equal', 'opportunity/affirmative', 'action'), 571),
 (('excellent', 'communication', 'skill'), 558),
 (('employer', '–', 'minority'), 556),
 (('big', 'data', 'tech

In [188]:
description_lemma

['we',
 'believe',
 'the',
 'answer',
 'are',
 'found',
 'when',
 'curious',
 ',',
 'courageous',
 'and',
 'collaborative',
 'people',
 'like',
 'you',
 'are',
 'brought',
 'together',
 'in',
 'an',
 'inspiring',
 'environment',
 '.',
 'where',
 'you',
 '’',
 're',
 'given',
 'opportunity',
 'to',
 'explore',
 'the',
 'power',
 'of',
 'digital',
 'and',
 'data',
 '.',
 'where',
 'you',
 '’',
 're',
 'empowered',
 'to',
 'risk',
 'failure',
 'by',
 'taking',
 'smart',
 'risk',
 ',',
 'and',
 'where',
 'you',
 '’',
 're',
 'surrounded',
 'by',
 'people',
 'who',
 'share',
 'your',
 'determination',
 'to',
 'tackle',
 'the',
 'world',
 '’',
 's',
 'toughest',
 'medical',
 'challenge',
 '.',
 'we',
 'are',
 'novartis',
 '.',
 'join',
 'u',
 'and',
 'help',
 'u',
 're-imagine',
 'medicine',
 '.',
 'two',
 'company',
 'and',
 'one',
 'incredible',
 'alliance',
 '.',
 'novartis',
 'and',
 'microsoft',
 'have',
 'formed',
 'alliance',
 'to',
 'leverage',
 'data',
 '&',
 'amp',
 ';',
 'artifici

In [264]:

df_descr = df['description']
n_doc = df.shape[0]


rep = {" ".join(t) : "-".join(t) for t, c in ngrams_3[0:500] }

# # use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in rep.items()) 
# #Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
pattern = re.compile("|".join(rep.keys()))
# text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
df_rep = df_descr.apply(lambda row: pattern.sub(lambda m: rep[re.escape(m.group(0))], str(row).lower()))


rep = {" ".join(t) : "-".join(t) for t, c in ngrams_2[0:1500] }
# # use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in rep.items()) 
# #Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
pattern = re.compile("|".join(rep.keys()))
# text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)

df_rep2 = df_rep.apply(lambda row: pattern.sub(lambda m: rep[re.escape(m.group(0))], str(row).lower()))



In [232]:
df_rep[100]

"the tableau specialist will work-closely with our internal-teams and external-clients in the creation, launch, maintenance, and enhancement of tableau dashboards. the goal is to aide internal and client-side analysts in deriving marketing insights from data through on-demand, live dashboards.\nresponsibilities include:\nfull lifecycle design and develop of tableau reports and dashboards\ncollaborate-closely with in-house development and database teams on data-driven product solutions\ntrain and educate co-workers on tableau tools and best-practices\nlead enhancement and product-development discussions to continually innovate reporting and analytics\ncoordinate multiple client requests and project activities at any one time to ensure accurate, timely and efficient reporting and analysis\ncreate custom dashboards as needed based on client requests and needs\nuser license and access management\nparticipate in the design and delivery of curriculum related to the rollout of new reports and

In [267]:
sum(df_rep2.apply(lambda row: "natural-language-processing" in row))

784

In [272]:
vectorizer = TfidfVectorizer()

docs_tfidf = vectorizer.fit_transform(df_rep2)

def get_tf_idf_query_similarity(vectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: cosine similarity between query and all docs
    """
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    return cosineSimilarities

In [273]:
test_resume = '''Malik Rabb

Seattle, WA • (123) 456-7891
mrabb@email.com
SUMMARY

Data Scientist with strong math background and 3+ years of experience using predictive modeling, data processing, and data mining algorithms to solve challenging business problems. Involved in Python open source community and passionate about deep reinforcement learning.
EDUCATION
Coral Springs University
Current - Current

Bachelor of Science in Mathematics
EXPERIENCE
River Tech, Data Scientist
Current - Current

    Built fuzzy matching algorithm using k-nearest neighbors to identify non-exact matching duplicates
    Designed and developed real time recommendation engine to rank sales leads for upsell opportunities
    Refined personalization algorithms for 1M+ customers on web and mobile
    Transformed raw data into MySQL with custom-made ETL application to prepare unruly data for machine learning

Retail Ocean, Data Scientist
Current - Current

    Leveraged 200M+ tweets to develop sentiment analysis model that helped improve sales and marketing strategies
    Used Python and Spark to scrape, clean, and analyze large datasets
    Helped build tools for detecting botnets with machine learning and data mining

SKILLS

    2nd place at Coral Springs Big Data Hackathon (out of 150+ participants)
    Java, Python, C++, Hadoop ecosystem, and MySQL
    Data cleansing, modeling, and mining
    Machine learning
'''

In [293]:
test_resume2 = '''Ivy Haddington

Austin, TX • (123) 456-7891
ihaddington@email.com
SUMMARY

Focused and quick-learning Software Engineer with 3 years of experience in computer science, programming, and UX design for various projects and clients.
EDUCATION
Hawaii Western
Aug '10 - May '14

Bachelor of Science in Electrical Engineering
EXPERIENCE
Crane & Jenkins, Software Engineer
Feb '15 - Current

    Integrate code changes, test code changes, and document code changes
    Developed Perl and PHP scripts to translate data between applications
    Developed automation scripts to test storage appliances in Python
    Practice agile development methodologies and work with current networking technologies
    Used Hadoop to build a scalable distributed data solution

Tradelot, Software Developer
Current - Current

    Wrote, modified, and debugged software for clients
    Developed code to complete assigned project by deadline
    Developed HTML, CSS, JavaScript, and JSP pages for user interaction
    Wrote Python scripts to load data and parse XML documents 

SKILLS

    Certified Scrum Master
    Cisco Certified Network Associate
    Cisco Certified Network Professional Security
    Unix and Linux
    Oracle
    SQL
    Java
    C++
    HTML
    '''

In [294]:
result2 = get_tf_idf_query_similarity(vectorizer, docs_tfidf, test_resume2)

In [296]:
df[['job_title', 'job_location', 'description']].iloc[result2.argsort()[::-1][0:100]]

,job_title,job_location,description
9553,Data Management Lead CI/CD,"Sterling, VA 20166","Sterling, VA 20166\n\nCulmen International is ..."
14286,Senior Applications Engineer JAVA Oracle Labs,"Redwood City, CA 94065",\n\nPreferred Qualifications\n\nJob Descriptio...
14312,Principal Applications Engineer JAVA Oracle ...,"Redwood City, CA 94065",\n\nPreferred Qualifications\n\nJob Descriptio...
10671,Software Engineer Level 3 (ID),"Annapolis Junction, MD",ProSync is seeking an experienced Software Eng...
6411,Software Developer,"Chantilly, VA 20151",\nThe Software Application Programmer will des...
...,...,...,...
7121,Cyber Security Data Scientist,"Fort Belvoir, VA 22060",\nJob Description:\n\nSummary: Global Resource...
8638,Mid Level Database Engineer,"Arlington, VA",www.vaxcom.com\nVaxcom Services Inc. (an Xator...
10161,Digital Analytics Implementation & QA Engineer,"El Segundo, CA 90245",\nPeople who Inspire Us.\n\nStories that Drive...
6401,Database Engineer (MariaDB),"Dulles, VA",www.vaxcom.com\nVaxcom Services Inc. (an Xator...


In [297]:
def n_matching_jobs(resume_str, vectorizer, docs_tfidf, n):
    similarities = get_tf_idf_query_similarity(vectorizer, docs_tfidf, resume_str)
    return df[['job_title', 'job_location', 'description']].iloc[similarities.argsort()[::-1][0:n]]

In [299]:
n_matching_jobs(test_resume2, vectorizer, docs_tfidf, 100)

,job_title,job_location,description
9553,Data Management Lead CI/CD,"Sterling, VA 20166","Sterling, VA 20166\n\nCulmen International is ..."
14286,Senior Applications Engineer JAVA Oracle Labs,"Redwood City, CA 94065",\n\nPreferred Qualifications\n\nJob Descriptio...
14312,Principal Applications Engineer JAVA Oracle ...,"Redwood City, CA 94065",\n\nPreferred Qualifications\n\nJob Descriptio...
10671,Software Engineer Level 3 (ID),"Annapolis Junction, MD",ProSync is seeking an experienced Software Eng...
6411,Software Developer,"Chantilly, VA 20151",\nThe Software Application Programmer will des...
...,...,...,...
7121,Cyber Security Data Scientist,"Fort Belvoir, VA 22060",\nJob Description:\n\nSummary: Global Resource...
8638,Mid Level Database Engineer,"Arlington, VA",www.vaxcom.com\nVaxcom Services Inc. (an Xator...
10161,Digital Analytics Implementation & QA Engineer,"El Segundo, CA 90245",\nPeople who Inspire Us.\n\nStories that Drive...
6401,Database Engineer (MariaDB),"Dulles, VA",www.vaxcom.com\nVaxcom Services Inc. (an Xator...


In [301]:
n_matching_jobs(test_resume, vectorizer, docs_tfidf, 100)

,job_title,job_location,description
8442,Senior Data Scientist,"Milwaukee, WI",\nRole Summary:\nThe Senior Data Scientist app...
3272,Data Architect,"Raleigh, NC 27609",\nPurpose:\n\nTo turn raw data into valuable i...
12138,Data Scientist,"Plano, TX 75023",\nThe Lead Data Scientist will support the suc...
4632,Associate Data Scientist/Data Scientist,"Allentown, PA 18101","\nPosition located in Allentown, PA or Washing..."
4664,Associate Data Scientist / Data Scientist,"Allentown, PA 18101",\nWe've got an electric future. The energy ind...
...,...,...,...
12073,Senior Data ScientistTechnology,"Dallas, TX 75215",A Senior Data Scientist is responsible for ana...
5031,Lead Engineer (Completions & Wellbore Interven...,"Houston, TX",\nWe are looking for a Lead Engineer who will ...
7700,Sr. Data & Applied Scientist,"Redmond, WA",\nWe are looking for a strong Senior Data Scie...
3088,Senior Data Scientist,"Morrisville, NC 27560",Description\n\nThe Senior Data Scientist at PP...


In [302]:
vectorizer.get_feature_names()

['00',
 '000',
 '0000',
 '00000',
 '000000',
 '00001',
 '00001313',
 '00004374',
 '0000477',
 '0000482',
 '0000505317',
 '00007613',
 '0001',
 '00014681',
 '00014915',
 '0002',
 '00024616',
 '0004',
 '0005',
 '00075710',
 '000atms',
 '000ft2',
 '000s',
 '001',
 '0010',
 '00102682',
 '00105323',
 '00124',
 '001441',
 '00149',
 '001851',
 '001_mstarinc',
 '002',
 '0027190717',
 '0031190717',
 '0039',
 '004',
 '0056',
 '006',
 '00630',
 '0069',
 '00am',
 '00pm',
 '01',
 '0101',
 '0105',
 '011',
 '012020',
 '0126',
 '013',
 '0132',
 '0134534',
 '015',
 '0171191121',
 '01730',
 '0174',
 '0175',
 '0183',
 '01m',
 '01t00',
 '02',
 '020',
 '0201',
 '0203',
 '0204',
 '021',
 '02110',
 '02111',
 '02114',
 '02115',
 '0212191120',
 '02129',
 '02144',
 '02145',
 '0216191121',
 '0225c8aa5f07e0cc2a3726cb054eced2253f9fc2',
 '0239',
 '02451',
 '0250190723',
 '0271',
 '02711',
 '02817',
 '03',
 '030',
 '0301',
 '0304',
 '03079',
 '0337190723',
 '0343',
 '037',
 '0382191122',
 '0399',
 '04',
 '040',
 '05